In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Set file paths (Make sure you place your files correctly in your Google Drive)
crop_path = '/content/drive/MyDrive/Colab Notebooks/crop_fertilizer_dataset/crop_recommendation.csv'
fertilizer_path = '/content/drive/MyDrive/Colab Notebooks/crop_fertilizer_dataset/fertitlizer_recommendation.csv'


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [4]:
# Load datasets
crop_df = pd.read_csv(crop_path)
fert_df = pd.read_csv(fertilizer_path)


In [5]:
# Display first 5 rows of each dataset
print("\nCrop Recommendation Dataset:")
print(crop_df.head())

print("\nFertilizer Recommendation Dataset:")
print(fert_df.head())



Crop Recommendation Dataset:
    N   P   K  temperature   humidity        ph    rainfall label
0  90  42  43    20.879744  82.002744  6.502985  202.935536  rice
1  85  58  41    21.770462  80.319644  7.038096  226.655537  rice
2  60  55  44    23.004459  82.320763  7.840207  263.964248  rice
3  74  35  40    26.491096  80.158363  6.980401  242.864034  rice
4  78  42  42    20.130175  81.604873  7.628473  262.717340  rice

Fertilizer Recommendation Dataset:
   Temparature  Humidity   Moisture Soil Type  Crop Type  Nitrogen  Potassium  \
0           26         52        38     Sandy      Maize        37          0   
1           29         52        45     Loamy  Sugarcane        12          0   
2           34         65        62     Black     Cotton         7          9   
3           32         62        34       Red    Tobacco        22          0   
4           28         54        46    Clayey      Paddy        35          0   

   Phosphorous Fertilizer Name  
0            0    

In [6]:
# Dataset Info
print("\nCrop Dataset Info:")
print(crop_df.info())

print("\nFertilizer Dataset Info:")
print(fert_df.info())


Crop Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2200 entries, 0 to 2199
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   N            2200 non-null   int64  
 1   P            2200 non-null   int64  
 2   K            2200 non-null   int64  
 3   temperature  2200 non-null   float64
 4   humidity     2200 non-null   float64
 5   ph           2200 non-null   float64
 6   rainfall     2200 non-null   float64
 7   label        2200 non-null   object 
dtypes: float64(4), int64(3), object(1)
memory usage: 137.6+ KB
None

Fertilizer Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99 entries, 0 to 98
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Temparature      99 non-null     int64 
 1   Humidity         99 non-null     int64 
 2   Moisture         99 non-null     int64 
 3   Soil Type        99 non-nul

# Data Cleaning & Preprocessing


In [7]:
# Check for missing values
print("\nMissing values in Crop Dataset:")
print(crop_df.isnull().sum())

print("\nMissing values in Fertilizer Dataset:")
print(fert_df.isnull().sum())



Missing values in Crop Dataset:
N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

Missing values in Fertilizer Dataset:
Temparature        0
Humidity           0
Moisture           0
Soil Type          0
Crop Type          0
Nitrogen           0
Potassium          0
Phosphorous        0
Fertilizer Name    0
dtype: int64


In [8]:
# Encode categorical columns in fertilizer dataset
le = LabelEncoder()
if 'Fertilizer Name' in fert_df.columns:
    fert_df['Fertilizer Name'] = le.fit_transform(fert_df['Fertilizer Name'])
if 'Crop Type' in fert_df.columns:
    fert_df['Crop Type'] = le.fit_transform(fert_df['Crop Type'])
if 'Soil Type' in fert_df.columns:
    fert_df['Soil Type'] = le.fit_transform(fert_df['Soil Type'])

In [9]:
# Standardize crop dataset features
scaler = StandardScaler()
crop_features = crop_df.drop('label', axis=1)
crop_target = crop_df['label']
crop_scaled = scaler.fit_transform(crop_features)

In [10]:
# Encode crop target
crop_target_encoded = le.fit_transform(crop_target)

# Split crop dataset
X_crop_train, X_crop_test, y_crop_train, y_crop_test = train_test_split(
    crop_scaled, crop_target_encoded, test_size=0.2, random_state=42)

In [11]:
# Train a simple model on crop data
crop_model = RandomForestClassifier()
crop_model.fit(X_crop_train, y_crop_train)
y_crop_pred = crop_model.predict(X_crop_test)

print("\nCrop Recommendation Model Report:")
print(classification_report(y_crop_test, y_crop_pred))
print("Accuracy:", accuracy_score(y_crop_test, y_crop_pred))


Crop Recommendation Model Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        23
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        20
           3       1.00      1.00      1.00        26
           4       1.00      1.00      1.00        27
           5       1.00      1.00      1.00        17
           6       1.00      1.00      1.00        17
           7       1.00      1.00      1.00        14
           8       0.92      1.00      0.96        23
           9       1.00      1.00      1.00        20
          10       0.92      1.00      0.96        11
          11       1.00      1.00      1.00        21
          12       1.00      1.00      1.00        19
          13       1.00      0.96      0.98        24
          14       1.00      1.00      1.00        19
          15       1.00      1.00      1.00        17
          16       1.00      1.00      1.00   

In [12]:
# Fertilizer Dataset Preprocessing
fert_features = fert_df.drop('Fertilizer Name', axis=1)
fert_target = fert_df['Fertilizer Name']

fert_scaled = scaler.fit_transform(fert_features)

In [13]:
# Split fertilizer dataset
X_fert_train, X_fert_test, y_fert_train, y_fert_test = train_test_split(
    fert_scaled, fert_target, test_size=0.2, random_state=42)

In [14]:
# Train a simple model on fertilizer data
fert_model = RandomForestClassifier()
fert_model.fit(X_fert_train, y_fert_train)
y_fert_pred = fert_model.predict(X_fert_test)

In [15]:
print("\nFertilizer Recommendation Model Report:")
print(classification_report(y_fert_test, y_fert_pred))
print("Accuracy:", accuracy_score(y_fert_test, y_fert_pred))


Fertilizer Recommendation Model Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         5
           5       1.00      1.00      1.00         5
           6       1.00      1.00      1.00         6

    accuracy                           1.00        20
   macro avg       1.00      1.00      1.00        20
weighted avg       1.00      1.00      1.00        20

Accuracy: 1.0
